In [1]:
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
import os
import pprint

In [2]:
## set up

In [3]:
service = Service(r"C:\web driver\msedgedriver.exe")
df = pd.read_csv("Jupyter Notebook/tgdd/raw_data.csv")
links = df.link

In [4]:
driver = webdriver.Edge(service=service)
driver.get(df.link[0])
sleep(3) 
driver.quit()

In [5]:
# setup headless mode
options = webdriver.EdgeOptions()
options.add_argument("--headless=new")  
options.add_argument("--window-size=1920,1080") 

In [10]:
def crawl_links(links, service, limit=10):
    dfs = {}  
    rating_and_sales = []  
    i = 0
    data_id = []
    for link in links[:limit]:
        driver = webdriver.Edge(service=service, options=options)
        driver.get(link)

        specs = {}
        
        section = driver.find_element(By.CSS_SELECTOR, "section.detail.detailv2")
        pid = section.get_attribute("data-id")
        data_id.append(pid)
        
        # --- get specs ---
        boxes = driver.find_elements(By.CSS_SELECTOR, "div.box-specifi")
        for box in boxes:
            try:
                box.click()
                sleep(0.3)
            except:
                pass

            titles = box.find_elements(By.TAG_NAME, "h3")
            group_name = titles[0].text.strip() if titles else "Unknown"
            specs[group_name] = {}

            items = box.find_elements(By.CSS_SELECTOR, "ul.text-specifi li")
            for item in items:
                strongs = item.find_elements(By.TAG_NAME, "strong")
                key = strongs[0].text.strip().replace(":", "") if strongs else "Thông tin khác"

                values = item.find_elements(By.CSS_SELECTOR, "aside span, aside a")
                parts = [v.text.strip() for v in values if v.text.strip()]

                if parts:
                    value = parts if len(parts) > 1 else parts[0]
                else:
                    full_text = item.get_attribute("innerText").strip()
                    if strongs:
                        for s in strongs:
                            full_text = full_text.replace(s.text, "")
                        value = full_text.strip(" -:\n")
                    else:
                        value = full_text
                specs[group_name][key] = value

        # --- get sales & rating ---
        try:
            quantity_sales = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, "span.quantity-sale")]
            quantity_sales = quantity_sales[0] if quantity_sales else None
        except Exception:
            quantity_sales = None

        try:
            rating = driver.find_element(By.CSS_SELECTOR, "div.detail-rate p").text
        except Exception:
            rating = None

        driver.quit()

        # --- append df ---
        for group, values in specs.items():
            if group not in dfs:
                dfs[group] = pd.DataFrame()
            row = values.copy()
            row["data_id"] = pid
            dfs[group] = pd.concat([dfs[group], pd.DataFrame([row])], ignore_index=True)

        # --- append rating & sales ---
        rating_and_sales.append({
            "data_id" : pid, 
            "rating": rating,
            "quantity_sales": quantity_sales
        })
        print(f"crawl {i} succeed")
        i+= 1

    return dfs, rating_and_sales, data_id

In [11]:
dfs, rating_and_sales, data_id = crawl_links(links, service, limit=len(links))

crawl 0 succeed
crawl 1 succeed
crawl 2 succeed
crawl 3 succeed
crawl 4 succeed
crawl 5 succeed
crawl 6 succeed
crawl 7 succeed
crawl 8 succeed
crawl 9 succeed
crawl 10 succeed
crawl 11 succeed
crawl 12 succeed
crawl 13 succeed
crawl 14 succeed
crawl 15 succeed
crawl 16 succeed
crawl 17 succeed
crawl 18 succeed
crawl 19 succeed
crawl 20 succeed
crawl 21 succeed
crawl 22 succeed
crawl 23 succeed
crawl 24 succeed
crawl 25 succeed
crawl 26 succeed
crawl 27 succeed
crawl 28 succeed
crawl 29 succeed
crawl 30 succeed
crawl 31 succeed
crawl 32 succeed
crawl 33 succeed
crawl 34 succeed
crawl 35 succeed
crawl 36 succeed
crawl 37 succeed
crawl 38 succeed
crawl 39 succeed
crawl 40 succeed
crawl 41 succeed
crawl 42 succeed
crawl 43 succeed
crawl 44 succeed
crawl 45 succeed
crawl 46 succeed
crawl 47 succeed
crawl 48 succeed
crawl 49 succeed
crawl 50 succeed
crawl 51 succeed
crawl 52 succeed
crawl 53 succeed
crawl 54 succeed
crawl 55 succeed
crawl 56 succeed
crawl 57 succeed
crawl 58 succeed
crawl 5

In [12]:
dfs.keys()

dict_keys(['Bộ xử lý', 'Đồ hoạ (GPU)', 'Bộ nhớ RAM, Ổ cứng', 'Màn hình', 'Cổng kết nối & tính năng mở rộng', 'Kích thước - Khối lượng - Pin'])

In [13]:
dfs['Bộ xử lý'].head()

,Công nghệ CPU,Số nhân,Số luồng,Tốc độ CPU,data_id,NPU,Hiệu năng xử lý AI (TOPS),
0,"[AMD Ryzen 5 - 7430U, AMD Ryzen 5]",6,12,2.30 GHz (Lên đến 4.30 GHz khi tải nặng),327098,NaN,NaN,NaN
1,"[AMD Ryzen 5 - 7520U, AMD Ryzen 5]",4,8,2.80 GHz (Lên đến 4.30 GHz khi tải nặng),311178,NaN,NaN,NaN
2,Intel Core i5 Raptor Lake - 13420H,8,12,2.10 GHz (Lên đến 4.60 GHz khi tải nặng),334442,NaN,NaN,NaN
3,Apple M4 - Hãng không công bố,10,Hãng không công bố,120 GB/s memory bandwidth,335362,NaN,NaN,NaN
4,"[Intel Core i5 Alder Lake - 1235U, Intel Core ...",10,12,1.30 GHz (Lên đến 4.40 GHz khi tải nặng),337470,NaN,NaN,NaN


In [14]:
dfs['Đồ hoạ (GPU)'].head()

,Card màn hình,data_id,Số nhân GPU,Công suất đồ hoạ - TGP,Hiệu năng xử lý AI (TOPS)
0,"[Card tích hợp - AMD Radeon Graphics, Card tíc...",327098,NaN,NaN,NaN
1,"[Card tích hợp - AMD Radeon Graphics, Card tíc...",311178,NaN,NaN,NaN
2,"[Card tích hợp - Intel UHD Graphics, Card tích...",334442,NaN,NaN,NaN
3,"[Card tích hợp - 8 nhân GPU, Card tích hợp]",335362,NaN,NaN,NaN
4,"[Card tích hợp - Intel Iris Xe Graphics, Card ...",337470,NaN,NaN,NaN


In [15]:
dfs['Bộ nhớ RAM, Ổ cứng'].head()

,RAM,Loại RAM,Tốc độ Bus RAM,Hỗ trợ RAM tối đa,Ổ cứng,data_id
0,16 GB,DDR4 (1 khe 8 GB + 1 khe 8 GB),3200 MHz,16 GB,512 GB SSD NVMe PCIe,327098
1,16 GB,LPDDR5 (Onboard),5500 MHz,Không hỗ trợ nâng cấp,"512 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...",311178
2,16 GB,DDR5 (1 khe 8 GB onboard + 1 khe 8 GB),4800 MHz,24 GB,"[512 GB SSD NVMe PCIe, Hỗ trợ khe cắm SSD M.2]",334442
3,16 GB,Hãng không công bố,Hãng không công bố,Không hỗ trợ nâng cấp,256 GB SSD,335362
4,16 GB,DDR4 (1 khe 8 GB + 1 khe 8 GB),2600 MHz,16 GB,"512 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...",337470


In [16]:
dfs['Màn hình'].head()

,Kích thước màn hình,Độ phân giải,Tần số quét,Độ phủ màu,Công nghệ màn hình,data_id,Tấm nền,Màn hình cảm ứng
0,"15.6""",Full HD (1920 x 1080),60Hz,45% NTSC,"[Chống chói Anti Glare, 250 nits]",327098,NaN,NaN
1,"15.6""",Full HD (1920 x 1080),60Hz,45% NTSC,"[Chống chói Anti Glare, 250 nits]",311178,TN,NaN
2,"15.3""",WUXGA (1920 x 1200),60Hz,45% NTSC,"[Chống chói Anti Glare, 300 nits]",334442,IPS,NaN
3,"13.6""",Liquid Retina (2560 x 1664),Hãng không công bố,NaN,"[Wide color (P3), 500 nits brightness, True To...",335362,IPS,NaN
4,"15.6""",Full HD (1920 x 1080),120Hz,45% NTSC,"[Chống chói Anti Glare, LED Backlit, 250 nits,...",337470,NaN,NaN


In [17]:
dfs['Cổng kết nối & tính năng mở rộng'].head()

,Cổng giao tiếp,Kết nối không dây,Webcam,Đèn bàn phím,Bảo mật,Công nghệ âm thanh,Tản nhiệt,data_id,Tính năng khác,Khe đọc thẻ nhớ
0,"[2 x USB Type-A, HDMI, 1 x USB Type-C (chỉ hỗ ...","[Wi-Fi 6 (802.11ax), Bluetooth 5.3]",HD webcam,Không có đèn,Công tắc khoá camera,Loa kép (2 kênh),Hãng không công bố,327098,NaN,NaN
1,"[1 x USB 2.0, Jack tai nghe 3.5 mm, 1 x USB 3....","[Wi-Fi 6E (802.11ax), Bluetooth 5.3]",HD webcam,Không có đèn,"[Công tắc khoá camera, Bảo mật vân tay]",SonicMaster audio,Hãng không công bố,311178,"[Độ bền chuẩn quân đội MIL STD 810H, Bản lề mở...",NaN
2,"[USB Type-C (hỗ trợ truyền dữ liệu, Power Deli...","[Bluetooth 5.2, Wi-Fi 6 (802.11ax)]",HD webcam,Không có đèn,"[TPM 2.0, Công tắc khoá camera]",Dolby Audio,Hãng không công bố,334442,Độ bền chuẩn quân đội MIL STD 810H,SD
3,"[Jack tai nghe 3.5 mm, MagSafe 3, 2 x Thunderb...","[Wi-Fi 6E (802.11ax), Bluetooth 5.3]",1080p FaceTime HD camera,Đơn sắc - Màu trắng,Bảo mật vân tay,"[3 microphones, 4 Loa, Dolby Atmos]",Hãng không công bố,335362,NaN,NaN
4,"[1 x USB 2.0, 2 x USB 3.2, HDMI, 1 x Headphone...","[Bluetooth 5.2, Wi-Fi 802.11 a/b/g/n/ac]",HD webcam,Có,NaN,Realtek Audio,NaN,337470,NaN,SD


In [18]:
dfs['Kích thước - Khối lượng - Pin'].head()

,Thông tin Pin,Hệ điều hành,Thời điểm ra mắt,Kích thước,Chất liệu,data_id
0,"3-cell Li-ion, 41 Wh",Windows 11 Home SL,2023,Dài 359.8 mm - Rộng 236 mm - Dày 18.6 mm - 1.5...,Vỏ nhựa,327098
1,"3-cell Li-ion, 42 Wh",Windows 11 Home SL,2023,Dài 360.3 mm - Rộng 232.5 mm - Dày 17.9 mm - 1...,Vỏ nhựa,311178
2,60 Wh,Windows 11 Home SL,2024,Dài 343.4 mm - Rộng 239.5 mm - Dày 17.9 mm - 1...,Vỏ nhựa - nắp lưng bằng kim loại,334442
3,"Li-Po, 53.8 Wh",macOS Sequoia,03/2025,Dài 304.1 mm - Rộng 215 mm - Dày 11.3 mm - 1.2...,Vỏ nhôm tái chế 100%,335362
4,"3-cell Li-ion, 41 Wh",Windows 11 Home SL + Office Home 2024 vĩnh viễn,2025,Dài 358.5 mm - Rộng 235.56 mm - Dày 18.99 mm -...,Vỏ nhựa,337470


In [19]:
data_id

['327098',
 '311178',
 '334442',
 '335362',
 '337470',
 '336967',
 '321192',
 '338207',
 '334792',
 '334450',
 '333427',
 '326124',
 '289472',
 '334803',
 '335556',
 '338323',
 '326050',
 '334996',
 '331362',
 '325500',
 '340564',
 '338311',
 '323920',
 '334997',
 '335372',
 '326279',
 '310449',
 '340486',
 '334446',
 '339679',
 '333430',
 '328235',
 '321467',
 '337233',
 '328237',
 '337420',
 '334999',
 '338204',
 '329786',
 '341563',
 '326830',
 '339683',
 '333886',
 '332365',
 '333104',
 '311176',
 '341612',
 '343120',
 '335554',
 '325247',
 '325691',
 '328238',
 '338206',
 '327872',
 '337476',
 '326049',
 '331564',
 '326122',
 '343121',
 '339687',
 '338278',
 '338324',
 '337474',
 '311177',
 '322072',
 '328906',
 '310447',
 '343122',
 '339681',
 '340560',
 '341270',
 '330162',
 '338318',
 '318163',
 '339857',
 '335553',
 '328814',
 '327707',
 '331566',
 '341607',
 '327050',
 '340559',
 '336036',
 '325697',
 '334998',
 '328949',
 '337042',
 '326831',
 '327097',
 '335555',
 '333103',

In [24]:
rating_and_sales

[{'data_id': '327098', 'rating': '4.9', 'quantity_sales': 'Đã bán 13,4k'},
 {'data_id': '311178', 'rating': '4.9', 'quantity_sales': 'Đã bán 23,5k'},
 {'data_id': '334442', 'rating': '4.9', 'quantity_sales': 'Đã bán 3,2k'},
 {'data_id': '335362', 'rating': '4.9', 'quantity_sales': 'Đã bán 4,2k'},
 {'data_id': '337470', 'rating': '5', 'quantity_sales': 'Đã bán 2,2k'},
 {'data_id': '336967', 'rating': '5', 'quantity_sales': 'Đã bán 1,3k'},
 {'data_id': '321192', 'rating': '4.9', 'quantity_sales': 'Đã bán 13,4k'},
 {'data_id': '338207', 'rating': '5', 'quantity_sales': 'Đã bán 1,7k'},
 {'data_id': '334792', 'rating': '5', 'quantity_sales': 'Đã bán 3k'},
 {'data_id': '334450', 'rating': '4.9', 'quantity_sales': 'Đã bán 1,4k'},
 {'data_id': '333427', 'rating': '5', 'quantity_sales': 'Đã bán 1,4k'},
 {'data_id': '326124', 'rating': '4.9', 'quantity_sales': 'Đã bán 2,2k'},
 {'data_id': '289472', 'rating': '4.9', 'quantity_sales': 'Đã bán 8,3k'},
 {'data_id': '334803', 'rating': '5', 'quantity

In [29]:
def save_tocsv(dfs, rating_and_sales):
    # --- make ---
    os.makedirs("output_specs", exist_ok=True)

    # --- specs group ---
    for group, df in dfs.items():
        name = group.strip()
        df.to_csv(f"output_specs/{name}.csv", index=False)
        print(f"saving {name}.csv successful")

    # --- rating & sales ---
    pd.DataFrame(rating_and_sales).to_csv(f"output_specs/rating_sales.csv", index=False)

    print("Saving Successful")

In [30]:
save_tocsv(dfs, rating_and_sales)

saving Bộ xử lý.csv successful
saving Đồ hoạ (GPU).csv successful
saving Bộ nhớ RAM, Ổ cứng.csv successful
saving Màn hình.csv successful
saving Cổng kết nối & tính năng mở rộng.csv successful
saving Kích thước - Khối lượng - Pin.csv successful
Saving Successful
